<a href="https://colab.research.google.com/github/clairesarraille/airbnb_price_prediction/blob/main/03_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Bank

In [152]:
# Code Bank:
"""
drop_for_pipeline = ['desc_polarity', 'desc_subjectivity']
df.drop(drop_for_pipeline, axis=1, inplace=True)


"""

"\ndrop_for_pipeline = ['desc_polarity', 'desc_subjectivity']\ndf.drop(drop_for_pipeline, axis=1, inplace=True)\n\n\n"

# Import Packages

In [43]:
# Core Packages:
import pandas as pd
from pandas import DataFrame
import pickle
import numpy as np
from numpy import unique
from numpy import arange
from matplotlib import pyplot
import re
import os
import warnings
warnings.filterwarnings('ignore')

#"""
# Google Colab:
### Print Dataframe like Spreadsheet!
from google.colab import data_table
data_table.enable_dataframe_formatter()
#"""

#"""
# Mount Drive:
from google.colab import drive
drive.mount('/content/drive')
#"""

## Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Pre-processing Libraries:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

## Modeling Libraries:
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import textblob
from textblob import TextBlob

## Validation Libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import log_loss

## Warning Libraries
# Hides warnings
#import warnings
#from pandas.core.common import SettingWithCopyWarning
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
#warnings.filterwarnings(action='ignore',module='sklearn')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#df_pristine = pd.read_pickle("/content/drive/MyDrive/capstone_data/ready_for_modeling.pkl")
#df = df_pristine.copy()

# NLP Pre-Processing: Polarity and Subjectivity on `description`, `neighborhood_overview` and `host_about` columns

## Description

In [3]:
"""
df['description'] = df['description'].fillna('')
df['desc_polarity'] = df['description'].apply(lambda text: TextBlob(text).polarity)
df['desc_subjectivity'] = df['description'].apply(lambda text: TextBlob(text).subjectivity)
"""

## Neighborhood Overview

In [5]:
"""
df['neighborhood_overview'] = df['neighborhood_overview'].fillna('')
df['neigh_over_polarity'] = df['neighborhood_overview'].apply(lambda text: TextBlob(text).polarity)
df['neigh_over_subjectivity'] = df['neighborhood_overview'].apply(lambda text: TextBlob(text).subjectivity)
"""

## Host About

In [6]:
"""
df['host_about'] = df['host_about'].fillna('')
df['host_about_polarity'] = df['host_about'].apply(lambda text: TextBlob(text).polarity)
df['host_about_subjectivity'] = df['host_about'].apply(lambda text: TextBlob(text).subjectivity)
"""

## Pickled df having desc. polarity and subjectivity

In [7]:
#df.to_pickle('desc_nlp_done.pkl')

## Read in pickled dataframe that has NLP columns for description:

In [11]:
# Read in dataframe with NLP column for description:
df_nlp_done = pd.read_pickle("/content/drive/MyDrive/capstone_data/desc_nlp_done.pkl")

In [12]:
df = df_nlp_done.copy()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274958 entries, 0 to 274957
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   description              274958 non-null  object 
 1   neighborhood_overview    274958 non-null  object 
 2   host_about               274958 non-null  object 
 3   host_has_profile_pic     274958 non-null  int64  
 4   host_identity_verified   274958 non-null  int64  
 5   latitude                 274958 non-null  float64
 6   longitude                274958 non-null  float64
 7   property_type            274958 non-null  object 
 8   room_type                274958 non-null  object 
 9   accommodates             274958 non-null  int64  
 10  bedrooms                 205357 non-null  float64
 11  beds                     272084 non-null  float64
 12  price                    274958 non-null  float64
 13  minimum_nights           274958 non-null  int64  
 14  maxi

# Create df for pipeline:
- Drop:
  - Free text columns we already converted to polarity and subjectivity:
    - `'description', 'neighborhood_overview', 'host_about'`
  - `amenities_keep` (the list of amenities we already transformed to boolean columns

In [14]:
df = df.drop(['description', 'neighborhood_overview', 'host_about', 'amenities_keep'], axis=1)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274958 entries, 0 to 274957
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   host_has_profile_pic     274958 non-null  int64  
 1   host_identity_verified   274958 non-null  int64  
 2   latitude                 274958 non-null  float64
 3   longitude                274958 non-null  float64
 4   property_type            274958 non-null  object 
 5   room_type                274958 non-null  object 
 6   accommodates             274958 non-null  int64  
 7   bedrooms                 205357 non-null  float64
 8   beds                     272084 non-null  float64
 9   price                    274958 non-null  float64
 10  minimum_nights           274958 non-null  int64  
 11  maximum_nights           274958 non-null  int64  
 12  instant_bookable         274958 non-null  int64  
 13  city                     274958 non-null  object 
 14  stat

# Inspect Columns to OHE

## Categorical Columns:

In [ ]:
data_table.enable_dataframe_formatter()
cat_cols = []
for col in range(df.shape[1]):
  if df[df.columns[col]].dtypes == 'object':
    cat_cols.append(df.columns[col])
print("Categorical Columns")
print()
for col in cat_cols:
  print(col)
cat_df = df[cat_cols]
print()
cat_df.apply(lambda cat_df: cat_df.sample(5, random_state=0))

In [39]:
# There are no nulls in the categorical columns
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274958 entries, 0 to 274957
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   property_type  274958 non-null  object
 1   room_type      274958 non-null  object
 2   city           274958 non-null  object
 3   state          274958 non-null  object
dtypes: object(4)
memory usage: 8.4+ MB


In [ ]:
DataFrame(df['property_type'].value_counts())

In [ ]:
DataFrame(df['room_type'].value_counts())

In [ ]:
DataFrame(df['city'].value_counts())

In [ ]:
DataFrame(df['state'].value_counts())

## Boolean Columns:

In [37]:
bool_cols = []
for col in range(df.shape[1]):
  nunique_vals = df.iloc[:,col].nunique()
  if nunique_vals == 2:
    bool_cols.append(df.columns[col])
print("Boolean Columns")
print()
for col in bool_cols:
  print(col)
bool_df = df[bool_cols]
print()
bool_df.apply(lambda bool_df: bool_df.sample(5, random_state=0))

Boolean Columns

host_has_profile_pic
host_identity_verified
instant_bookable
license_bool
is_shared_bathroom
air_conditioning
kitchen
wifi
free_park_prem
pets_allowed
pool



,host_has_profile_pic,host_identity_verified,instant_bookable,license_bool,is_shared_bathroom,air_conditioning,kitchen,wifi,free_park_prem,pets_allowed,pool
114282,1,1,1,0,0,1,1,1,1,0,1
119428,1,1,0,0,0,1,1,1,0,0,0
269296,1,1,1,0,0,1,0,1,0,0,1
196869,1,1,0,0,0,1,1,1,1,0,0
215411,1,1,1,1,0,1,1,1,1,1,0


In [38]:
# There are no nulls in the boolean columns:
df[bool_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274958 entries, 0 to 274957
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   host_has_profile_pic    274958 non-null  int64
 1   host_identity_verified  274958 non-null  int64
 2   instant_bookable        274958 non-null  int64
 3   license_bool            274958 non-null  int64
 4   is_shared_bathroom      274958 non-null  int64
 5   air_conditioning        274958 non-null  int64
 6   kitchen                 274958 non-null  int64
 7   wifi                    274958 non-null  int64
 8   free_park_prem          274958 non-null  int64
 9   pets_allowed            274958 non-null  int64
 10  pool                    274958 non-null  int64
dtypes: int64(11)
memory usage: 23.1 MB


## Numeric Columns (Excluding `price` and `bool_cols`)

In [28]:
num_cols = []
for col in range(df.shape[1]):
  if df[df.columns[col]].dtypes != 'object' and df.columns[col] not in bool_cols:
    num_cols.append(df.columns[col])
num_cols.remove('price')
print("Numeric Columns")
print()
for col in num_cols:
  print(col)
num_df = df[num_cols]
print()
num_df.apply(lambda num_df: num_df.sample(5, random_state=0))

Numeric Columns

latitude
longitude
accommodates
bedrooms
beds
minimum_nights
maximum_nights
bathrooms
desc_polarity
desc_subjectivity
neigh_over_polarity
neigh_over_subjectivity
host_about_polarity
host_about_subjectivity



,latitude,longitude,accommodates,bedrooms,beds,minimum_nights,maximum_nights,bathrooms,desc_polarity,desc_subjectivity,neigh_over_polarity,neigh_over_subjectivity,host_about_polarity,host_about_subjectivity
114282,43.155015,-77.589279,2,1.0,1.0,1,365,1.0,0.220714,0.416429,0.000000,0.000000,0.000000,0.000000
119428,40.680720,-73.982490,2,1.0,1.0,30,1125,1.0,0.111588,0.312163,0.395623,0.477273,0.000000,0.000000
269296,36.110140,-115.171743,2,1.0,1.0,1,28,1.0,0.334091,0.521970,0.000000,0.000000,0.000000,0.000000
196869,30.261490,-97.717400,5,NaN,1.0,1,1125,1.0,0.155073,0.474403,0.234127,0.573016,0.566667,0.466667
215411,44.905330,-93.327470,8,3.0,3.0,2,1125,2.0,0.405000,0.632500,0.500000,0.666667,0.318182,0.527273


In [42]:
# There are nulls in bedrooms, beds, bathrooms, but relatively few
# We'll use a simple imputer for the numeric col nulls
df[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274958 entries, 0 to 274957
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   latitude                 274958 non-null  float64
 1   longitude                274958 non-null  float64
 2   accommodates             274958 non-null  int64  
 3   bedrooms                 205357 non-null  float64
 4   beds                     272084 non-null  float64
 5   minimum_nights           274958 non-null  int64  
 6   maximum_nights           274958 non-null  int64  
 7   bathrooms                274616 non-null  float64
 8   desc_polarity            274958 non-null  float64
 9   desc_subjectivity        274958 non-null  float64
 10  neigh_over_polarity      274958 non-null  float64
 11  neigh_over_subjectivity  274958 non-null  float64
 12  host_about_polarity      274958 non-null  float64
 13  host_about_subjectivity  274958 non-null  float64
dtypes: f

# Add Intercept to Data:
- We can avoid biased residuals if we add an intercept
- https://medium.com/datascienceray/why-add-an-intercept-column-into-datasets-a2385cdb33bc

In [81]:
df['intercept'] = 1

In [82]:
df['intercept'].value_counts()

1    274958
Name: intercept, dtype: int64

# Define Modeling Class ModelWithCV()

In [83]:
class ModelWithCV():
    '''Structure to save the model and more easily see its crossvalidation'''

    def __init__(self, model, model_name, X, y, cv_now=True):
        self.model = model
        self.name = model_name
        self.X = X
        self.y = y
        # For CV results
        self.cv_results = None
        self.cv_mean = None
        self.cv_median = None
        self.cv_std = None
        #
        if cv_now:
            self.cross_validate()

    def cross_validate(self, X=None, y=None, kfolds=5):
        '''
        Perform cross-validation and return results.

        Args:
          X:
            Optional; Training data to perform CV on. Otherwise use X from object
          y:
            Optional; Training data to perform CV on. Otherwise use y from object
          kfolds:
            Optional; Number of folds for CV (default is 10)
        '''

        cv_X = X if X else self.X
        cv_y = y if y else self.y

        # MSE Results:
        self.cv_mse_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds, scoring ='neg_mean_squared_error')
        self.cv_mse_mean = np.mean(self.cv_mse_results)
        self.cv_mse_median = np.median(self.cv_mse_results)
        self.cv_mse_std = np.std(self.cv_mse_results)

        # MAE Results:
        self.cv_mae_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds, scoring ='neg_mean_absolute_error')
        self.cv_mae_mean = np.mean(self.cv_mae_results)
        self.cv_mae_median = np.median(self.cv_mae_results)
        self.cv_mae_std = np.std(self.cv_mae_results)

        # RMSE Results:
        self.cv_rmse_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds, scoring ='neg_root_mean_squared_error')
        self.cv_rmse_mean = np.mean(self.cv_rmse_results)
        self.cv_rmse_median = np.median(self.cv_rmse_results)
        self.cv_rmse_std = np.std(self.cv_rmse_results)

        # R2 Results:
        self.cv_r2_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds)
        self.cv_r2_mean = np.mean(self.cv_r2_results)
        self.cv_r2_median = np.median(self.cv_r2_results)
        self.cv_r2_std = np.std(self.cv_r2_results)

    def print_cv_summary(self):
        cv_summary = (
        f'''CV Results for `{self.name}` model:
            R-Squared: {self.cv_r2_mean:.5f} ± {self.cv_r2_std:.5f}
            Root Mean Squared Error: {self.cv_rmse_mean} ± {self.cv_rmse_std:.5f}
            Mean Absolute Error: {self.cv_mae_mean} ± {self.cv_mae_std:.5f}
            Mean Squared Error: {self.cv_mse_mean} ± {self.cv_mse_std:.5f}
            10-fold Cross Validation Scores: {self.cv_r2_results}
        ''')
        print(cv_summary)

# Train Test Split

## First Split

In [84]:
# Create X (predictors) and y (target) variables:
X_1 = df.drop('price', axis=1)
y_1 = df['price']

# Split Data into train and test:
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, random_state=666)

## Second Split

In [85]:
# Split Data into train and test:
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_1_train, y_1_train, random_state=666)


# Pipeline Set-up

### Boolean features don't need to be scaled
- so we will use 'remainder='passthrough'' To indicate to compiler to let them through pipeline without transformations.

In [86]:
print(cat_cols)

['property_type', 'room_type', 'city', 'state']


In [87]:
print(bool_cols)

['host_has_profile_pic', 'host_identity_verified', 'instant_bookable', 'license_bool', 'is_shared_bathroom', 'air_conditioning', 'kitchen', 'wifi', 'free_park_prem', 'pets_allowed', 'pool']


In [88]:
print(num_cols)

['latitude', 'longitude', 'accommodates', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'bathrooms', 'desc_polarity', 'desc_subjectivity', 'neigh_over_polarity', 'neigh_over_subjectivity', 'host_about_polarity', 'host_about_subjectivity']


## Pipeline for numerics:

In [89]:
# Pipeline for numerics (SCALE data):

numpipe = Pipeline([
    ('stan_scale', StandardScaler()),
    ('num_impute', SimpleImputer(strategy='median'))
])


## Pipeline for Categoricals:

In [90]:
# Pipeline for categorical feature (One-Hot-Encode):

catpipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore')) # Using handle_unknown param en lieu of stratify during train-test-split
])

## We now have our numeric and categorical pipelines.
- Since we don't have any nulls, and binary features don't need to be transformed or one-hot-encoded,
- Our binary features don't need any transformations, so we will use `remainder='passthrough'` to let them through.
- Next step is to handle columns holistically with `ColumnTransformer`

In [91]:
# We will use our pipeline objects as transformer argument for ColumnTransformer
# our transformer is a tuple and includes a columns argument

ColTrans = ColumnTransformer(transformers=[
    ('numerics', numpipe, num_cols),
    ('categoricals', catpipe, cat_cols)
], remainder='passthrough')

# Model #1: Baseline Dummy

In [92]:
# Create Dummy Model Pipeline

dumpipe = Pipeline([
    ('ct', ColTrans),
    ('dummy', DummyRegressor(strategy='mean'))
])

In [93]:
# Fit the dummy regressor to the training data:

dumpipe.fit(X_2_train, y_2_train)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerics',
                                                  Pipeline(steps=[('stan_scale',
                                                                   StandardScaler()),
                                                                  ('num_impute',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['latitude', 'longitude',
                                                   'accommodates', 'bedrooms',
                                                   'beds', 'minimum_nights',
                                                   'maximum_nights',
                                                   'bathrooms', 'desc_polarity',
                                                   'desc_subjectivity',
                                                   'neigh_over_polarity',
                                                   'neigh_over_subjectivity',
                                                   'host_about_polarity',
                                                   'host_about_subjectivity']),
                                                 ('categoricals',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['property_type', 'room_type',
                                                   'city', 'state'])])),
                ('dummy', DummyRegressor())])

## Dummy Regressor Cross validation on Training Data:

In [94]:

# Now let's instantiate a dummy pipeline model and fit via inheritance from the ModelWithCV class:
dum_pipe_cv = ModelWithCV(
    dumpipe,
    model_name='dummy',
    X=X_2_train,
    y=y_2_train,
)

In [95]:
dum_pipe_cv.print_cv_summary()

CV Results for `dummy` model:
            R-Squared: -0.00004 ± 0.00004
            Root Mean Squared Error: -951.4114265294854 ± 137.36254
            Mean Absolute Error: -219.10723721594718 ± 4.37318
            Mean Squared Error: -924052.1695826652 ± 257613.42217
            10-fold Cross Validation Scores: [-9.71921643e-07 -1.29942088e-05 -1.19020139e-04 -8.16565079e-06
 -3.91309529e-05]
        


## Dummy Regressor Results on Test Data:

In [63]:
#X_1_train, X_1_test, y_1_train, y_1_test
print(f'Dummy model TEST Mean Accuracy score: {dumpipe.score(X_1_test, y_1_test):.8f}')

Dummy model TEST Mean Accuracy score: -0.00000207
